# Crude Oil Challenge
## About
<p>This notebook presents my approach to the [crude oil challenge](https://challengedata.ens.fr/en/challenge/25/predict_the_crude_oil_production_trend.html#) proposed by Societe General.<br>
My current score on the leaderboard is 0.80, the regressor used being the RandomForestClassifier of sklearn.
<br><br>_NB:_<br>
I created a Github repository for this project, that provides a better insight into the structure of my work. [Check it out](https://github.com/Edouard360/crude-oil-ml) ! </p>
<br>
## First Look Into The Data

In [2]:
import pandas as pd

train_df = pd.read_csv("../data/train.csv",delimiter=";",header=0,index_col=0);
train_label = pd.read_csv("../data/label.csv",delimiter=";",header=0,index_col=0);

<p>What is the shape of my data ?</p>

In [3]:
train_df.shape

(10159, 122)

### Handling the column names
<p>We can identify columns groups from our data.</p>

In [4]:
columns_group = [train_df.columns[2+10*i:2+10*(i+1)] for i in range(12)]
columns_group.insert(0,train_df.columns[0:2])

<p>And that way we have</p>

In [5]:
columns_group[1]

Index(['1_diffClosing stocks(kmt)', '1_diffExports(kmt)', '1_diffImports(kmt)',
       '1_diffRefinery intake(kmt)', '1_diffWTI',
       '1_diffSumClosing stocks(kmt)', '1_diffSumExports(kmt)',
       '1_diffSumImports(kmt)', '1_diffSumProduction(kmt)',
       '1_diffSumRefinery intake(kmt)'],
      dtype='object')

In [6]:
columns_group[2]

Index(['2_diffClosing stocks(kmt)', '2_diffExports(kmt)', '2_diffImports(kmt)',
       '2_diffRefinery intake(kmt)', '2_diffWTI',
       '2_diffSumClosing stocks(kmt)', '2_diffSumExports(kmt)',
       '2_diffSumImports(kmt)', '2_diffSumProduction(kmt)',
       '2_diffSumRefinery intake(kmt)'],
      dtype='object')

<p>Let's create useful functions to __operate on these columns__. For instance, I'd like to be able to get:</p>

+ All the statistics for one period. Just like the columns_group[1] output.
+ All the statistics for many periods. The period set could be expressed like: `[1,2,3,4]` or `range(1,5)`.
+ The statistics for only one category ( like `'diffImports(kmt)'` ) over all the periods (period1 to period12).
+ The statistics for a list of categories ( like `['diffImports(kmt)','diffWTI','diffExports']` ) over all the periods.
+ The statistics for a list of categories ( like `['diffImports(kmt)','diffWTI','diffExports']` ) for some given periods ( like `range(4,8)` ).

<p>The following lines of code provide these features.</p>


In [4]:
COLUMNS_SUFFIX = ['_diffClosing stocks(kmt)',
                  '_diffExports(kmt)', '_diffImports(kmt)',
                  '_diffRefinery intake(kmt)', '_diffWTI',
                  '_diffSumClosing stocks(kmt)', '_diffSumExports(kmt)',
                  '_diffSumImports(kmt)', '_diffSumProduction(kmt)',
                  '_diffSumRefinery intake(kmt)']


def check_type_prefix(i):
    if (type(i) is int):
        i = [i]
    elif ((type(i) is not range) and (type(i) is not list)):
        raise TypeError("Only range, list or single integer allowed !")
    return i


def check_type_suffix(i):
    if (type(i) is str):
        i = [i]
    elif ((type(i) is not list)):
        raise TypeError("Only list or single string allowed !")
    return i


def get_prefix(i):
    i = check_type_prefix(i)
    return [str(j) + suffix for j in i for suffix in COLUMNS_SUFFIX]


def get_suffix(suffix_request, prefix=range(1, 13)):
    prefix = check_type_prefix(prefix)
    suffix_request = check_type_suffix(suffix_request)
    for i in range(len(suffix_request)):
        for original_suffix in COLUMNS_SUFFIX:
            if (str.lower(original_suffix).find(str.lower(suffix_request[i])) != -1):
                suffix_request[i] = original_suffix
                break
    suffix_request = list(set(suffix_request))
    return [str(j) + suffix for j in prefix for suffix in suffix_request]


<p>Now we can simply use ```get_prefix``` and ```get_suffix``` that way:</p>

+ All the statistics for one period. 

In [5]:
get_prefix(5)

['5_diffClosing stocks(kmt)',
 '5_diffExports(kmt)',
 '5_diffImports(kmt)',
 '5_diffRefinery intake(kmt)',
 '5_diffWTI',
 '5_diffSumClosing stocks(kmt)',
 '5_diffSumExports(kmt)',
 '5_diffSumImports(kmt)',
 '5_diffSumProduction(kmt)',
 '5_diffSumRefinery intake(kmt)']

+ All the statistics for many periods.

In [8]:
get_prefix(range(7,9))

['7_diffClosing stocks(kmt)',
 '7_diffExports(kmt)',
 '7_diffImports(kmt)',
 '7_diffRefinery intake(kmt)',
 '7_diffWTI',
 '7_diffSumClosing stocks(kmt)',
 '7_diffSumExports(kmt)',
 '7_diffSumImports(kmt)',
 '7_diffSumProduction(kmt)',
 '7_diffSumRefinery intake(kmt)',
 '8_diffClosing stocks(kmt)',
 '8_diffExports(kmt)',
 '8_diffImports(kmt)',
 '8_diffRefinery intake(kmt)',
 '8_diffWTI',
 '8_diffSumClosing stocks(kmt)',
 '8_diffSumExports(kmt)',
 '8_diffSumImports(kmt)',
 '8_diffSumProduction(kmt)',
 '8_diffSumRefinery intake(kmt)']

+ The statistics for only one category over all the periods. (Observe the leniency for the name of the feature).

In [6]:
get_suffix('Imports') # get_suffix('imports') and get_suffix('_diffImports(kmt)') give the same result.

['1_diffImports(kmt)',
 '2_diffImports(kmt)',
 '3_diffImports(kmt)',
 '4_diffImports(kmt)',
 '5_diffImports(kmt)',
 '6_diffImports(kmt)',
 '7_diffImports(kmt)',
 '8_diffImports(kmt)',
 '9_diffImports(kmt)',
 '10_diffImports(kmt)',
 '11_diffImports(kmt)',
 '12_diffImports(kmt)']

+ The statistics for a list of categories over all the periods. (Again, the function is lenient for the feature's names)

In [10]:
get_suffix(['Imports','Exports'])

['1_diffImports(kmt)',
 '1_diffExports(kmt)',
 '2_diffImports(kmt)',
 '2_diffExports(kmt)',
 '3_diffImports(kmt)',
 '3_diffExports(kmt)',
 '4_diffImports(kmt)',
 '4_diffExports(kmt)',
 '5_diffImports(kmt)',
 '5_diffExports(kmt)',
 '6_diffImports(kmt)',
 '6_diffExports(kmt)',
 '7_diffImports(kmt)',
 '7_diffExports(kmt)',
 '8_diffImports(kmt)',
 '8_diffExports(kmt)',
 '9_diffImports(kmt)',
 '9_diffExports(kmt)',
 '10_diffImports(kmt)',
 '10_diffExports(kmt)',
 '11_diffImports(kmt)',
 '11_diffExports(kmt)',
 '12_diffImports(kmt)',
 '12_diffExports(kmt)']

+ And finally, the statistics for a list of categories __for some given periods__.

In [12]:
get_suffix(['wti','refinery'],range(1,4))

['1_diffRefinery intake(kmt)',
 '1_diffWTI',
 '2_diffRefinery intake(kmt)',
 '2_diffWTI',
 '3_diffRefinery intake(kmt)',
 '3_diffWTI']

<p>Now that we have tools for handling the columns let's move on ! 
<br><br>_NB:_<br> There is also a ```except_suffix``` function (that does what its name suggest).
[Github link](https://github.com/Edouard360/crude-oil-ml/blob/master/tools/features_name.py) for the full code.</p>

## Dealing with missing values
<p>We can explicit the columns where the missing values are located :</p>

In [27]:
def discrimateColumns(train_df):
    describe_df = train_df.describe()
    columns_missing = train_df.columns[(describe_df.loc['count'] < len(train_df)).values]
    columns_full = train_df.columns[(describe_df.loc['count'] == len(train_df)).values]
    return (columns_missing,columns_full)
print("Columns where we find missing values :\n")
print(discrimateColumns(train_df)[0])
print("\nTotal number of missing values :\n")
print(train_df.isnull().sum().sum())

Columns where we find missing values :

Index(['1_diffClosing stocks(kmt)', '1_diffImports(kmt)',
       '2_diffClosing stocks(kmt)', '2_diffImports(kmt)',
       '3_diffClosing stocks(kmt)', '3_diffImports(kmt)',
       '4_diffClosing stocks(kmt)', '4_diffImports(kmt)',
       '5_diffClosing stocks(kmt)', '5_diffImports(kmt)',
       '6_diffClosing stocks(kmt)', '6_diffImports(kmt)',
       '7_diffClosing stocks(kmt)', '7_diffImports(kmt)',
       '8_diffClosing stocks(kmt)', '8_diffImports(kmt)',
       '9_diffClosing stocks(kmt)', '9_diffImports(kmt)',
       '10_diffClosing stocks(kmt)', '10_diffImports(kmt)',
       '11_diffClosing stocks(kmt)', '11_diffImports(kmt)',
       '12_diffClosing stocks(kmt)', '12_diffImports(kmt)'],
      dtype='object')

Total number of missing values :

4497


<p>We could impute values simply by __choosing the median for each column__.<br>
Or we could __group with respect to the country and the month__ and then __impute the median__. </p> <br>
```group = ["country","month"]
train_df.ix[:,2:] = \
    train_df.groupby(group,as_index=False).transform(lambda x: x.fillna(x.median()))[train_df.columns[2:]]
```
<br>
<p>Yet, is is also very common to __fill the NAs with a preliminary regression__. </p>

In [31]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

def fillRegression(train_df):
    columns_missing,columns_full = discrimateColumns(train_df)
    for target_column in columns_missing:
        #print(target_column) # Uncomment this to see the progression
        X = train_df[columns_full].values
        y = train_df[target_column].values
        train_is = np.arange(len(y))[~np.isnan(y)]
        test_is = np.arange(len(y))[np.isnan(y)]
        clf = RandomForestRegressor(n_estimators=10, max_leaf_nodes=5)
        clf.fit(X[train_is], y[train_is])
        train_df.ix[test_is, target_column] = clf.predict(X[test_is])
        columns_full = np.append(columns_full, target_column)
    return train_df

train_df = fillRegression(train_df)

<p>And we can now check that there are no missing values left:</p>

In [32]:
print("\nTotal number of missing values :\n")
print(train_df.isnull().sum().sum())


Total number of missing values :

0


### Dummy Coding

<p>The first group of features needs to be dummy coded:</p>

In [7]:
columns_group[0]

Index(['month', 'country'], dtype='object')

<p>This will add 12 entries for the month, and 76 for the countries, which gives a total of 120 + 12 + 76 = 208 features.</p>

In [8]:
train_df = pd.get_dummies(train_df, drop_first=False, columns=['month'])
train_df = pd.get_dummies(train_df, drop_first=False, columns=['country'])